In [14]:
from datasets import load_dataset

dataset = load_dataset("ccdv/arxiv-summarization")
dataset

No config specified, defaulting to: arxiv-summarization/section
Found cached dataset arxiv-summarization (C:/Users/dujus/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3)
100%|██████████| 3/3 [00:00<00:00,  6.51it/s]


DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6440
    })
})

In [15]:
from datasets.dataset_dict import DatasetDict

data_train = dataset['train']
data_test = DatasetDict({'validation': dataset['validation'], 'test': dataset['test']})

In [16]:
data_test

DatasetDict({
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6440
    })
})

In [17]:
from transformers import AutoTokenizer

checkpoint = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [18]:
def tokenize_function(data):
    return tokenizer(data['abstract'], data['article'], truncation=True, )

In [19]:
tok = data_test.map(tokenize_function, batched=True)

Loading cached processed dataset at C:/Users/dujus/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3\cache-9176a0aac702aaee.arrow
Loading cached processed dataset at C:/Users/dujus/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3\cache-1514b99dc5cb2b03.arrow


In [20]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
# samples = tok[:2000]
# samples = {k: v for k, v in samples.items() if k not in ['abstract', 'article']}
# batch = data_collator(samples)

KeyError: "Invalid key: slice(None, 2000, None). Please first select a split. For example: `my_dataset_dictionary['test'][slice(None, 2000, None)]`. Available splits: ['test', 'validation']"

In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [23]:
from transformers import AutoModel
model = AutoModel.from_pretrained(checkpoint)

In [24]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tok['test'],
    eval_dataset=['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [31]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartModel.forward` and have been ignored: article, abstract. If article, abstract are not expected by `BartModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 6440
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2415
  Number of trainable parameters = 406290432
  0%|          | 0/2415 [00:36<?, ?it/s]


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 33554432 bytes.

In [ ]:
# chunk = 500
# tok_abs = []

# for i in (data_test[pos:pos + chunk] for pos in range(0, len(data_test), chunk)):
#     tok_abs.append(tokenizer(i['abstract'], truncation=True, padding=True, return_tensors='tf'))


In [ ]:
# tok_art = []
# for i in (data_test[pos:pos + chunk] for pos in range(0, len(data_test), chunk)):
#     tok_art.append(tokenizer(i['article'], truncation=True, ))

In [ ]:
# import torch
# tok = torch.cat(tok_abs, dim=1)


In [ ]:
# test_tok_abs = tokenizer(data_train['article'], truncation=True, padding='max_length')